In [1]:
from threading import *
import time 
from datetime import datetime
from time import sleep
from datetime import timedelta

In [2]:
#import rpy2.robjects as robjects #r-python interfacing

import rpy2.robjects as robjects
from pymeasure.instruments import list_resources

import ftd2xx  # Thorlabs MFF101
import ftd2xx.defines as constants
from ftd2xx import listDevices, getDeviceInfoDetail






from newportxps import NewportXPS
from newportxps.XPS_C8_drivers import XPS, XPSException





import numpy as np
import csv


xpsd_remoteip = '192.168.254.254'
# xpsd_hostip ='192.168.0.254'
hostname = 'XPS-1b81'

xps = NewportXPS(xpsd_remoteip)
print(xps.status_report())


xps.kill_group('XYZ')
xps.initialize_allgroups()
xps.home_allgroups()



def pos_all():
    for sname, _ in xps.stages.items():
        print('{}: {}'.format(sname, xps.get_stage_position(sname)))
        
        




# XPS host:         192.168.254.254 (XPS-1B81)
# Firmware:         XPS-D-N13019
# Current Time:     Sun Mar 14 15:58:14 2021
# Last Reboot:      Thu Jan  7 10:21:03 2021
# Trajectory Group: XYZ
# Groups and Stages
XYZ (multipleaxesinuse), Status: Not initialized state due to an emergency brake : see positioner error
   XYZ.Pos1 (MFA@MFA-CC@XPS-DRV11)
      Hardware Status: ZM low level
      Positioner Errors: OK
   XYZ.Pos2 (MFA@MFA-CC@XPS-DRV11)
      Hardware Status: ZM low level
      Positioner Errors: OK
   XYZ.Pos3 (MFA@MFA-CC@XPS-DRV11)
      Hardware Status: ZM low level
      Positioner Errors: OK


# Initialize motion controller

In [3]:

#pos_all() tells about current position of the motion controller

pos_all()


xps.move_stage('XYZ.Pos1',10) #moves x axis
xps.move_stage('XYZ.Pos2',11) #moves y axis
xps.move_stage('XYZ.Pos3',8.03) #moves z axis

XYZ.Pos1: -8.8497e-05
XYZ.Pos2: -0.0001415952
XYZ.Pos3: -5.30982e-05


''

In [125]:
pos_all()

XYZ.Pos1: 9.4999936554
XYZ.Pos2: 10.499992056
XYZ.Pos3: 8.0300053872


# Initialize Spectrometer

In [4]:
from spectra import *
# from spectra import capture_photo,ration
capture_photo("start",2)

In [144]:
mirror('on')

{'type': 5, 'id': 67369712, 'description': b'APT Filter Flipper', 'serial': b'37000805'}


In [122]:
st=7.99
zz=[float(st)+(0.02*i) for i in range(4)]
zz

[7.99, 8.01, 8.03, 8.05]

In [123]:
st=7.97

zz=[float(st)+(0.02*i) for i in range(6)]
for i in range(len(zz)):
    xps.move_stage('XYZ.Pos3',zz[i])
    print("z axis at: ",zz[i])
    time.sleep(8)


z axis at:  7.97
z axis at:  7.989999999999999
z axis at:  8.01
z axis at:  8.03
z axis at:  8.049999999999999
z axis at:  8.07


# Adjust the z-axis

In [50]:
##calling the adjust() function will autofocus the z axis.

def adjust():
    line=1
    import timeit
#     begin=8.7
    start = timeit.default_timer()
    import time

    st=7.5

    zz=[float(st)+(0.1*i) for i in range(11)]
    intensity=[]
    for i in range(len(zz)):
        time.sleep(4)

        xps.move_stage('XYZ.Pos3',zz[i])
        time.sleep(1)
        p=capture_photo('adjust',i,line,1)
        intensity.append(p)
    mirror('off')
   # stop = timeit.default_timer()

    #print('Time: ', stop - start)  
    #print(intensity)

    import numpy as np
    ix=intensity.index(np.max(intensity))
    xps.move_stage('XYZ.Pos3',zz[ix])
    print("adjusted z position is: ",zz[ix])
    return zz[ix]

# Initialize Pressure


**current_pressure() reads the current pressure inside the chamber.** 

**open_valve('valve_name') is used to release gas from valve named as valve_name**

**valve_name= vac , out. vac stands for vaccum, out stands for outlet valve. so if you want to release the outlet then type --> open_valve('out')**

**To set the pressure inside the chamber, use gopr(pressure_value) function, suppose you need to set the pressure to 510 psi, then write the code --> gopr(510)**

**To close all the valves type -> close_all()**

**To close vac valve, type -> close_valve('vac')**

In [37]:
from pressure import close_valve,open_valve,close_all,current_pressure,gopr

current_pressure()

121

# Initialize Laser

laser_power.power=0.01 # sets the power to 1 mW

In [38]:
from pymeasure.instruments.lighthousephotonics import Sprout
laser_power = Sprout('COM4')
laser_power.adapter.connection.baud_rate = 19200
laser_power.adapter.connection.read_termination = '\r'
laser_power.adapter.connection.write_termination = '\r'
laser_power.power=0.01

C:\Users\UWAdmin\Anaconda3\lib\site-packages\pyvisa\highlevel.py:3355: FutureWarning: get_instrument is deprecated and will be removed in 1.12, use open_resource instead.
  FutureWarning,


# Initialize Mirror

In [40]:

from flipper import *

mirror("on")
# mirror("off")

{'type': 5, 'id': 67369712, 'description': b'APT Filter Flipper', 'serial': b'37000805'}


In [61]:
pos_all()

XYZ.Pos1: 9.0000033048
XYZ.Pos2: 10.499992056
XYZ.Pos3: 8.1800078022


# Directory things

In [59]:
import os
os.getcwd()
# os.mkdir('task1')
os.chdir(r'task1')

# Pattern a spot


In [101]:
print("Enter the x,y position: ")

x,y=[x for x in input().split()]

print("Enter the power in mW")
power_i=int(input())

print("Enter the time of irradiation in ms")
time_i=int(input())

print("enter spot number")
i=int(input())

print("Enter the pressure in psi")
pressure_i=int(input())

for i in range(3):gopr(pressure_i)

Enter the x,y position: 
9.5 11
Enter the power in mW
50
Enter the time of irradiation in ms
1000
enter spot number
4
Enter the pressure in psi
120
129
128
128
129
129
128
129
129
128
129
129
128
128
128
128
127
128
128
127
127
127
126
127
127
126
126
126
126
125
126
126
125
125
125
124
124
125
124
123
124
123
123
123
122
122
122
122
121
121
121
121
done
done
122
122
123
122
123
123
122
123
123
122
122
123
122
122
122
122
121
122
121
121
121
121
120
done


In [109]:
#move to the position where the patterning will take place
# xps.move_stage('XYZ.Pos1',x) #moves x axis
# xps.move_stage('XYZ.Pos2',y) #moves y axis
# xps.move_stage('XYZ.Pos3',z) #moves z axis


##check the spectra for precursor
# mirror("on")
# GD=capture_photo("before",i) #i is the spot number
# mirror("off")


####irradiation begins below
laser_power.power=((power_i-2.7979)/0.4639)/1000


time.sleep(25) #allow the power to reach it's value and get stabilized

mirror("on") #irradiation starts
time.sleep(time_i/1000) # mirror is open until irradiation period
mirror("off")
laser_power.power=0.01 # restore laser power to default



###now patterning is done. next step is to get the raman spectra

# mirror("on") 
# p=capture_photo("after",i)

# print("GD ratio is: ",p)

#For getting fitted curves, go to curve fit analysis lines and spots.ipynb file


{'type': 5, 'id': 67369712, 'description': b'APT Filter Flipper', 'serial': b'37000805'}
{'type': 5, 'id': 67369712, 'description': b'APT Filter Flipper', 'serial': b'37000805'}


In [78]:
time.sleep(time_i/1000)
print('hello')

hello


In [70]:
power_i=100
laser_power.power=((power_i-2.7979)/0.4639)/1000

In [108]:
laser_power.power=0.01

In [113]:
mirror('on')

{'type': 5, 'id': 67369712, 'description': b'APT Filter Flipper', 'serial': b'37000805'}


In [41]:
i=2
p=capture_photo("after",i)

print("GD ratio is: ",p)

 
 
patterning not done
GD ratio is:  0.0


In [29]:
df=pd.DataFrame(columns=['power','time','pressure','ratio'])
df['power'].loc[i]=power_i
df['time'].loc[i]=time_i
df['pressure'].loc[i]=120
df['ratio'].loc[i]=p

In [31]:
df.to_csv('task1.csv')

In [201]:
mirror("on") #irradiation starts
time.sleep(10000/1000) # mirror is open until irradiation period
mirror("off")
laser_power.power=0.01

{'type': 5, 'id': 67369712, 'description': b'APT Filter Flipper', 'serial': b'37000805'}
{'type': 5, 'id': 67369712, 'description': b'APT Filter Flipper', 'serial': b'37000805'}


In [203]:
mirror('off')

{'type': 5, 'id': 67369712, 'description': b'APT Filter Flipper', 'serial': b'37000805'}


In [202]:
mirror('on')

{'type': 5, 'id': 67369712, 'description': b'APT Filter Flipper', 'serial': b'37000805'}


In [207]:
close_all()

# open_valve('out')

current_pressure()

15